## Data Loading
Loads in the big data matrix and the stfp matrix created by matlab

In [1]:
from scipy.io import loadmat

def load_in_data():
    file = scipy.io.loadmat("data/simulationData.mat")
    data = file["data"]
    return data

def load_stfp():
    file = scipy.io.loadmat("data/stfp.mat")
    stfp = file["stfp"]
    return stfp


## Data processing
Processes the data just as in the matlab version

In [22]:
# Set the notebook directory as the working directory
%pwd
%cd C:\Users\Rafael\Documents\GitHub\BeyondHulten

C:\Users\Rafael\Documents\GitHub\BeyondHulten


In [4]:
import numpy as np
import pandas as pd
import scipy

#data is the IO matrix for 88 sectors over 46 years (1960-2005)
data2 = load_in_data()
ind = pd.MultiIndex.from_arrays(data2[:, (0,1)].T, names=('year','sec'))
data = pd.DataFrame(data2[:, 2:93], index=ind)
del data2, ind

In [5]:
#Column 0 of datavector contains gross output for each sector
print(data.loc[(1960,2),0]*2) #first column times two
print(data.loc[(1960,2)].sum()) #summed across all columns (including the sum in column 0 already)
#Column 1 and 2 contain labour and capital
print(data.loc[(2005,2),0:2])
#Columns 3 to 90 contain the 88 sectors
#Colum 91 contains noncompetitive imports


3434.026
3433.7749999999996
0    88366.023
1    25623.402
2    31151.107
Name: (2005.0, 2.0), dtype: float64


In [24]:
#delete 10 sectors
removable_sectors = [59] + list(range(79, 88))
data2 = data.drop(index=removable_sectors, level=1)

#key aggregates
grossy = data2.loc[:,0].unstack().T
capital = data2.loc[:,1].unstack().T
labor = data2.loc[:,2].unstack().T
vadd = labor + capital

#remaining sector list
sec = list(range(1,89))
for i in removable_sectors:
    sec.remove(i)
len(sec)

#delete sectors which don't produce anything
grossy.loc[(grossy==0).all(axis=1)] #sectors 8 and 62
zeroprod = list(grossy.loc[(grossy==0).all(axis=1)].index.values)

grossy = grossy.drop(index=zeroprod)
capital = capital.drop(index=zeroprod)
labor = labor.drop(index=zeroprod)
vadd = vadd.drop(index=zeroprod)

stfp = load_stfp()
Σ = np.cov(stfp, rowvar=False)
μ = np.mean(stfp, axis=1)

## Creating usable variables
Function that creates the metrics for a given year

In [56]:
def get_variables(year):
    IO = data2.loc[year,0]
    IO = IO.drop(index=zeroprod)
    IO.sum()
    Ω = IO / IO.sum()
    α = vadd[year]/grossy[year]
    β = (np.eye(len(grossy)) - np.diag(1 - α) @ Ω).T @ grossy[year]
    β[β < 0] = 0
    β = β / np.sum(β)
    λ = np.linalg.inv( np.eye(len(grossy)) - np.diag(1 - α) @ Ω).T @ β
    L = λ * α

    return α, β, Ω, L, λ

## Elasticities

In [58]:
ε = .5;
θ = 0.001;
σ = .9;

# Construction Site Below HERE

## Objective Function

Here the objective function in `SimulationDerivs.m` is formulated, also the Jacobian is given, to helpt the solver.

In [59]:
def problem(X, A, β, Ω, α, ε, θ, σ, L):
    N = len(α)
    p = X[:N]
    y = X[N:]

    Out = np.zeros(2 * N, dtype=X.dtype)

    q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
    w = p * (A ** ((ε - 1) / ε)) * (α ** (1 / ε)) * (y ** (1 / ε)) * L ** (-1 / ε)
    C = np.dot(w, L)

    Out[:N] = p - (A ** (ε - 1) * (α * w ** (1 - ε) + (1 - α) * q ** (1 - ε))) ** (1 / (1 - ε))
    Out[N:] = y - np.dot(np.dot(np.dot(np.dot(np.dot(y.T, np.diag(p) ** ε), np.diag(A) ** (ε - 1)), np.diag(q) ** (θ - ε)), np.diag(1 - α)), np.dot(np.dot(Ω, np.diag(p) ** (-θ)), β)) - np.dot(β.T, np.diag(p) ** (-σ) * C)

    return Out


## Generating Random Shocks

In [60]:
from scipy.stats import multivariate_normal

def generate_random_shock(Σ, α, Ω, λ):
    n = len(Σ)
    cov_matrix = -0.5 * np.diag(Σ)
    A = np.exp(multivariate_normal.rvs(mean=np.zeros(n), cov=cov_matrix))
    
    inv_term = np.linalg.inv(np.eye(n) - np.diag(1 - α) @ Ω)
    log_A = np.log(A)
    init = np.concatenate((np.exp(-inv_term @ log_A), λ / np.exp(-inv_term @ log_A)))
    
    return A, init


## Solving for shocks

In [61]:
trials = 100
GDP = np.zeros(trials)
λ_sim = np.zeros((76, trials))
α, β, Ω, L, λ = get_variables(1973) 


In [46]:
#This is from ChatGPT

import multiprocessing as mp
from scipy.optimize import NonlinearFunction, NonlinearProblem, root

# Parallel worker function
def parallel_worker(k):
    A, init = generate_random_shock(Σ, α, Ω, λ)
    p = [A, β, Ω, α, ε, θ, σ, L]
    prob = NonlinearProblem(lambda u: problem(u, *p), jac=lambda u: problem_jacobian(u, *p))
    sol = root(prob, init, method='hybr')

    x = np.real(sol.x)
    p = x[:76]
    q = x[76:]
    gdp = np.dot(p * (A ** ((ε - 1) / ε)) * (α ** (1 / ε)) * (q ** (1 / ε)) * (L ** (-1 / ε)), L)
    λ_sim[:, k] = (p * q) / gdp
    return gdp

# Parameters
trials = 100
GDP = np.zeros(trials)
λ_sim = np.zeros((76, trials))

α, β, Ω, L, λ = getVariables(1973)
Σ = np.array(...)  # Replace with appropriate values for Σ

# Parallel execution
num_processes = mp.cpu_count()
with mp.Pool(processes=num_processes) as pool:
    results = pool.map(parallel_worker, range(trials))
    GDP = np.array(results)

# Now you have both GDP and λ_sim populated after the parallel execution

ImportError: cannot import name 'NonlinearFunction' from 'scipy.optimize' (C:\Users\Rafael\Anaconda3\envs\Jupyter_plugin\lib\site-packages\scipy\optimize\__init__.py)